# Imports

In [ ]:
import pandas as pd
import os
from datetime import datetime
import numpy as np
import re
import configparser

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)

now = datetime.now().date()

# Create obj config and load paths
config = configparser.ConfigParser()
config.read('config.ini')
paths = config['paths']

# Treatments

In [ ]:
df_details = pd.read_csv(
    paths["path_investidor10_fiis_details"],
    sep=",")
df_details = df_details[
    [
        "fii_name", "quote", "dividend_yield", 
        "price_book_ratio", "liquidity", "appreciation_12months", 
        "number_unit_holders", "vacancy", "fund_type", "net_asset_value", 
        "management_fee", "issued_units", "market_sector", "net_asset_value_per_unit"
    ]
]

df_details = df_details.rename(columns={"fii_name": "Ativo"})
df_details = df_details.rename(columns={"quote": "Preco Atual"})
df_details = df_details.rename(columns={"dividend_yield": "DY"}) # Porcentagem em Dividendos
df_details = df_details.rename(columns={"price_book_ratio": "P/VP"}) # Preco sobre Valor Patrimonial
df_details = df_details.rename(columns={"liquidity": "Negociacao diaria"})
df_details = df_details.rename(columns={"appreciation_12months": "Variacao 12M"})
df_details = df_details.rename(columns={"number_unit_holders": "N/Cotistas"})
df_details = df_details.rename(columns={"vacancy": "Vacancia"})
df_details = df_details.rename(columns={"fund_type": "Tipo Fundo"})
df_details = df_details.rename(columns={"net_asset_value": "Valor Patrimonial"})
df_details = df_details.rename(columns={"management_fee": "Taxa Gerencia"})
df_details = df_details.rename(columns={"issued_units": "Cotas Emitidas"})
df_details = df_details.rename(columns={"market_sector": "Setor"})
df_details = df_details.rename(columns={"net_asset_value_per_unit": "VP/C"}) # Valor Parimonial por Cota

# Removed null rows (Fiis not found data in website) 
df_details = df_details.dropna(subset=['Preco Atual', 'DY', 'Negociacao diaria'], axis=0)

df_details['Preco Atual'] = df_details['Preco Atual'].str.replace("-", "0")
df_details['DY'] = df_details['DY'].str.replace("-", "0")
df_details['P/VP'] = df_details['P/VP'].str.replace("-", "0")
df_details['Negociacao diaria'] = df_details['Negociacao diaria'].str.replace("-", "0")
df_details['Variacao 12M'] = df_details['Variacao 12M'].str.replace("-", "0")
df_details['N/Cotistas'] = df_details['N/Cotistas'].str.replace("-", "0")
df_details['Vacancia'] = df_details['Vacancia'].str.replace("-", "0")
df_details['Valor Patrimonial'] = df_details['Valor Patrimonial'].str.replace("-", "0")

df_details["Ativo"] = df_details["Ativo"].str.upper()
df_details["Preco Atual"] = df_details["Preco Atual"].str.replace("R$ ", "").str.replace(".", "").str.replace(",", ".")
df_details["DY"] = df_details["DY"].str.replace("%", "").str.replace(",", ".")
df_details["P/VP"] = df_details["P/VP"].str.replace(".", "").str.replace(",", ".")
df_details["Negociacao diaria"] = df_details["Negociacao diaria"].str.replace("R$ ", "")
df_details["Variacao 12M"] = df_details["Variacao 12M"].str.replace("%", "").str.replace(".", "").str.replace(",", ".")
df_details["N/Cotistas"] = df_details["N/Cotistas"].astype("str").str.replace(".", "")
df_details['Vacancia'] = df_details['Vacancia'].astype("str").str.replace("%", "").str.replace(".", "").str.replace(",", ".")
df_details['Valor Patrimonial'] = df_details['Valor Patrimonial'].str.replace("R$ ", "")

def replace_m_k_values(row):
    if 'M' in row['Negociacao diaria']:
        row["Negociacao diaria"] = row["Negociacao diaria"].replace(" M", "0000").replace(",", "") 
    elif 'K' in row['Negociacao diaria']:
        row["Negociacao diaria"] = row["Negociacao diaria"].replace(" K", "0").replace(",", "")
    else:
        row["Negociacao diaria"] = row["Negociacao diaria"].replace(".", "").replace(",", ".")
    return row
df_details = df_details.apply(replace_m_k_values, axis=1)

def convert_to_scientific(value):
    numeric_value = float(re.sub(r'[^0-9.]', '', value))
    
    if 'Bilhões' in value or 'Bilhão' in value:
        numeric_value *= 1e9
    elif 'Mil' in value:
        if 'Milhões' in value or 'Milhão' in value:
            numeric_value *= 1e6
        else:
            numeric_value *= 1e3
    
    return f"{numeric_value:e}"

# Aplica a função de conversão para notação científica
df_details['Valor Patrimonial'] = df_details['Valor Patrimonial'].apply(convert_to_scientific)


def taxa_gerencia(texto):
    if not texto:
        texto = 0
    else: 
        texto = str(texto).strip(' ')
        texto = texto.split("%")[0]
        texto = texto.split("a.a.")[0]
        texto = texto.split("de")[0]
        try:
            texto = float(texto.replace(',', '.'))
        except:
            texto = 0
    return texto
df_details["Taxa Gerencia"] = df_details["Taxa Gerencia"].str.replace(" ", "")
df_details['Taxa Gerencia'] = df_details['Taxa Gerencia'].apply(taxa_gerencia)

df_details["Preco Atual"] = df_details["Preco Atual"].astype("float")
df_details["P/VP"] = df_details["P/VP"].astype("float")
df_details["DY"] = df_details["DY"].astype("float")
df_details["Negociacao diaria"] = df_details["Negociacao diaria"].astype("float")
df_details["Variacao 12M"] = df_details["Variacao 12M"].astype("float")
df_details["N/Cotistas"] = df_details["N/Cotistas"].astype("int")
df_details["Vacancia"] = df_details["Vacancia"].astype("float")
df_details['Valor Patrimonial'] = df_details['Valor Patrimonial'].astype("float")
df_details['Taxa Gerencia'] = df_details['Taxa Gerencia'].astype("float")

In [ ]:
df_properties = pd.read_csv(
    paths["path_investidor10_fiis_properties"],
    sep=",")

df_properties = df_properties.rename(columns={"fii_name": "Ativo"})

df_count = df_properties['Ativo'].value_counts().rename('Quant. Imoveis').reset_index()

df_fiis = df_details.merge(df_count, on='Ativo', how='left')

df_fiis['Quant. Imoveis'] = df_fiis['Quant. Imoveis'].fillna(0)
df_fiis['Quant. Imoveis'] = df_fiis['Quant. Imoveis'].astype('int')

# Score Padrao

In [ ]:
df_fiis['Score Padrao'] = 0

# Crie uma lista de condições
condicoes = [
    (df_fiis['P/VP'] <= 1.10) & (df_fiis['P/VP'] >= 0.90),
    df_fiis['Negociacao diaria'] >= 600000,
    (df_fiis['DY'] >= 8) & (df_fiis['DY'] <= 13),
    df_fiis['N/Cotistas'] >= 50000,
    df_fiis['Vacancia'] <= 10,
    df_fiis['Valor Patrimonial'] >= 1e6,
    df_fiis['Taxa Gerencia'] <= 1,
    df_fiis['Tipo Fundo'].isin(['Fundo de Tijolo', 'Fundo de fundos', 'Fundo misto']), # Tijolo = Segurança, FOF = Diversificação, Fundo = Equilibrio Volatil
    (df_fiis['Variacao 12M'] >= -5) & (df_fiis['Variacao 12M'] <= 5),
    df_fiis['Quant. Imoveis'] >= 5
]

# Percorra as condições e some os pontos
for i, condicao in enumerate(condicoes):
    df_fiis['Score Padrao'] += condicao

# Score Extra

In [ ]:
df_fiis['Vacancia Score'] = 0
df_fiis['Negociacao diaria Score'] = 0
df_fiis['N/Cotistas Score'] = 0
df_fiis['DY Score'] = 0
df_fiis['P/VP Score'] = 0
df_fiis['Quant. Imoveis Score'] = 0
df_fiis['Score Final'] = 0

# Obtenha os índices dos 5 menores valores de "Vacância"
df_fiis.loc[df_fiis['Vacancia'].nsmallest(10).index, 'Vacancia Score'] = 1

# Obtenha os índices dos 5 menores valores de "Negociacao diaria"
df_fiis.loc[df_fiis['Negociacao diaria'].nlargest(10).index, 'Negociacao diaria Score'] = 1

# Obtenha os índices dos 5 menores valores de "N/Cotistas"
df_fiis.loc[df_fiis['N/Cotistas'].nlargest(10).index, 'N/Cotistas Score'] = 1

# Selecione apenas as linhas que estão dentro da faixa de DY (7 a 12)
# Calcule a média e atribui 1 ponto às 5 linhas mais próxima da média
filtered_df = df_fiis[(df_fiis['DY'] >= 8) & (df_fiis['DY'] <= 13)]
dividend_yield_mean = filtered_df['DY'].mean()
df_fiis.loc[(filtered_df['DY'] - dividend_yield_mean).abs().nsmallest(5).index, 'DY Score'] = 1

# Obtenha os índices dos 10 maiores valores
# com patrimonio acima de 1 Bilhão e P/VP proximo a 1
filtered_df = df_fiis[(df_fiis['Valor Patrimonial'] >= 1e6) & (df_fiis['P/VP'] >= 0.95) & (df_fiis['P/VP'] <= 1.00)]
df_fiis.loc[filtered_df['Valor Patrimonial'].nlargest(10).index, 'P/VP Score'] = 1

# Obtem apenas os ativos que contem mais de 10 Imoveis listados, como garantia de diversificação
df_fiis.loc[df_fiis['Quant. Imoveis'] >= 10, 'Quant. Imoveis Score'] = 1

df_fiis['Score Final'] += df_fiis['Score Padrao']
df_fiis['Score Final'] += df_fiis['Vacancia Score']
df_fiis['Score Final'] += df_fiis['Negociacao diaria Score']
df_fiis['Score Final'] += df_fiis['N/Cotistas Score']
df_fiis['Score Final'] += df_fiis['DY Score']
df_fiis['Score Final'] += df_fiis['Quant. Imoveis Score']
df_fiis['Score Final'] += df_fiis['P/VP Score']

# Conclusão

In [ ]:
df_fiis = df_fiis.sort_values(['Score Final', 'Score Padrao'], ascending=False)
df_fiis[['Score Padrao', 'Score Final', 'Ativo', 'Preco Atual', 'VP/C', 'P/VP']].head(20)

In [ ]:
# df_fiis.head(20)